In [1]:
print('get qualified txs')
import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
# import flipside_utils as f
import clickhouse_utils as ch
import csv_utils as cu
# NOTE: google_bq_utils imported later (before BQ write) to avoid stale OIDC token in CI
import opstack_metadata_utils as ops
sys.path.pop()

import time

import numpy as np
import pandas as pd
import os
import clickhouse_connect as cc

get qualified txs


In [2]:
# d = ops.get_op_stack_metadata_by_data_source('flipside')
chain_configs = ops.generate_op_stack_chain_config_query_list()

# Get All Chain IDs in our metadata list & DB
# Commented out to read from Flip & CH responses
# chain_ids_string = ops.gen_chain_ids_list_for_param(chain_configs['mainnet_chain_id'])

# display(chain_configs)

In [3]:
ch_client = ch.connect_to_clickhouse_db() #Default is OPLabs DB

query_name = 'daily_evms_qualified_txs_counts'

In [4]:
col_list = [
        'dt','blockchain','name','layer','chain_id'
        , 'num_raw_txs', 'num_success_txs','num_qualified_txs'
        , 'sum_raw_gas_used', 'sum_success_gas_used','sum_qualified_gas_used'
        ,'source'
        ]

In [5]:
trailing_days = 28
chunk_size = 3

# flipside_configs = chain_configs[chain_configs['source'] == 'flipside']
clickhouse_configs = chain_configs[chain_configs['source'] == 'oplabs']

In [6]:
# print('     flipside runs')
# flip_dfs = []

# with open(os.path.join("inputs/sql/flipside_bychain.sql"), "r") as file:
#     og_query = file.read()

# for index, chain in flipside_configs.iterrows():
#     print('     flipside: ' + chain['blockchain'])
#     query = og_query
#     # Pass in Params to the query
#     query = query.replace("@blockchain@", chain['blockchain'])
#     query = query.replace("@chain_id@", str(chain['mainnet_chain_id']))
#     query = query.replace("@name@", chain['display_name'])
#     query = query.replace("@layer@", chain['chain_layer'])
#     query = query.replace("@trailing_days@", str(trailing_days))
    
#     try:
#         df = f.query_to_df(query)
#         flip_dfs.append(df)
#     except Exception as e:  # Use FlipsideError if available instead of Exception
#         print(f"Error querying Flipside for {chain['blockchain']}: {str(e)}")
#         print("Skipping this chain due to API credit limitation or other issues.")
#         continue

# if flip_dfs:
#     flip = pd.concat(flip_dfs)
#     flip['source'] = 'flipside'
#     flip['dt'] = pd.to_datetime(flip['dt']).dt.tz_localize(None)
#     flip = flip[col_list]
# else:
#     print("No data was retrieved from Flipside. The resulting DataFrame will be empty.")
#     flip = pd.DataFrame(columns=col_list)

In [7]:
# Run Clickhouse
print('     clickhouse runs')
ch_dfs = []
failed_chains = []  # Track failed chains for debugging
with open(os.path.join("inputs/sql/goldsky_bychain.sql"), "r") as file:
                        og_query = file.read()

for index, chain in clickhouse_configs.iterrows():
        trailing_days_remaining = trailing_days

        while trailing_days_remaining > 0:
                days_end = trailing_days_remaining
                days_start = max( trailing_days_remaining - chunk_size, 0)
                print(     'clickhouse: ' + chain['blockchain'] + ' - day range: ' + str(days_start) + ' - ' + str(days_end))
                query = og_query
                #Pass in Params to the query
                query = query.replace("@blockchain@", chain['blockchain'])
                query = query.replace("@name@", chain['display_name'])
                query = query.replace("@layer@", chain['chain_layer'])
                query = query.replace("@trailing_days_end@", str(days_end))
                query = query.replace("@trailing_days_start@", str(days_start))
                # print(query)
                try:
                        df = ch_client.query_df(query)
                        ch_dfs.append(df)
                except Exception as e:
                        error_msg = str(e)
                        print(f'ERROR processing {chain["blockchain"]} (days {days_start}-{days_end}): {error_msg[:200]}')
                        failed_chains.append({
                            'chain': chain['blockchain'],
                            'days_start': days_start,
                            'days_end': days_end,
                            'error': error_msg[:500]
                        })
                
                trailing_days_remaining -= chunk_size

# Summary of failures
if failed_chains:
    print(f'\n=== {len(failed_chains)} chunk(s) failed ===')
    for f in failed_chains:
        print(f"  - {f['chain']} (days {f['days_start']}-{f['days_end']}): {f['error'][:100]}...")

     clickhouse runs
clickhouse: zora - day range: 25 - 28
clickhouse: zora - day range: 22 - 25
clickhouse: zora - day range: 19 - 22
clickhouse: zora - day range: 16 - 19
clickhouse: zora - day range: 13 - 16
clickhouse: zora - day range: 10 - 13
clickhouse: zora - day range: 7 - 10
clickhouse: zora - day range: 4 - 7
clickhouse: zora - day range: 1 - 4
clickhouse: zora - day range: 0 - 1
clickhouse: base - day range: 25 - 28
clickhouse: base - day range: 22 - 25
clickhouse: base - day range: 19 - 22
clickhouse: base - day range: 16 - 19
clickhouse: base - day range: 13 - 16
clickhouse: base - day range: 10 - 13
clickhouse: base - day range: 7 - 10
clickhouse: base - day range: 4 - 7
clickhouse: base - day range: 1 - 4
clickhouse: base - day range: 0 - 1
clickhouse: mode - day range: 25 - 28
clickhouse: mode - day range: 22 - 25
clickhouse: mode - day range: 19 - 22
clickhouse: mode - day range: 16 - 19
clickhouse: mode - day range: 13 - 16
clickhouse: mode - day range: 10 - 13
click

In [8]:
if ch_dfs:
    ch = pd.concat(ch_dfs)
    ch['source'] = 'goldsky'
    ch['dt'] = pd.to_datetime(ch['dt']).dt.tz_localize(None)
    ch = ch[col_list]
else:
    print("No data was retrieved from Goldsky. The resulting DataFrame will be empty.")
    ch = pd.DataFrame(columns=col_list)

In [9]:
# Get Chains we already have data for & don't need to run in Dune
dataframes = [ch]#, flip]

chain_ids_string = ops.get_unique_chain_ids_from_dfs(dataframes)

185,252,7560,65536,53302,6805,177,7777777,254,1750,957,291,60808,42220,183,1868,360,57073,8008,1923,7897,5371,34443,130,10,8453,480,1135


In [10]:
ch.sample(5)

,dt,blockchain,name,layer,chain_id,num_raw_txs,num_success_txs,num_qualified_txs,sum_raw_gas_used,sum_success_gas_used,sum_qualified_gas_used,source
1,2026-01-26,orderly,Orderly,L2,291,10753,10678,8694,4987279208,4984456707,4908892028,goldsky
1,2026-01-13,worldchain,World,L2,480,1585692,1581816,1248798,664300060992,663848951533,533842628073,goldsky
1,2026-01-31,automata,Automata,L2,65536,104,104,24,336574614,336574614,25216268,goldsky
2,2026-01-20,race,Race,L2,6805,57,57,41,5495942,5495942,4993000,goldsky
0,2026-01-16,lisk,Lisk,L2,1135,12507,12275,6423,8306101174,8269251914,3711598068,goldsky


In [11]:
dune_df = pd.DataFrame()

In [ ]:
try:# Run Dune with chunking (similar to ClickHouse pattern)
    print('     dune runs')
    dune_dfs = []
    trailing_days_remaining = trailing_days
    
    while trailing_days_remaining > 0:
        days_end = trailing_days_remaining
        days_start = max(trailing_days_remaining - chunk_size, 0)
        lookback_days = trailing_days - days_end  # How far back from today to start
        chunk_trailing_days = days_end - days_start  # How many days in this chunk
        
        print(f'     dune: day range {days_start}-{days_end} (lookback_days={lookback_days}, trailing_days={chunk_trailing_days})')
        
        days_param = d.generate_query_parameter(input=chunk_trailing_days, field_name='trailing_days', dtype='number')
        lookback_param = d.generate_query_parameter(input=lookback_days, field_name='lookback_days', dtype='number')
        chain_ids_param = d.generate_query_parameter(input=chain_ids_string, field_name='list_chain_ids', dtype='text')

        chunk_df = d.get_dune_data(query_id = 3740822, #https://dune.com/queries/3740822
            name = "dune_evms_qualified_txs",
            path = "outputs",
            performance="medium",
            params = [days_param, lookback_param, chain_ids_param],
            num_hours_to_rerun=16
        )
        
        if not chunk_df.empty:
            dune_dfs.append(chunk_df)
        
        trailing_days_remaining -= chunk_size
    
    # Combine all chunks
    if dune_dfs:
        dune_df = pd.concat(dune_dfs, ignore_index=True)
    else:
        dune_df = pd.DataFrame()
        
except Exception as e:
    print(f"Error occurred, skipping: {e}")
    dune_df = pd.DataFrame()

     dune runs
     dune: day range 25-28 (lookback_days=0, trailing_days=3)
Results available at https://dune.com/queries/3740822?trailing_days=3&lookback_days=0&list_chain_ids=185%2C252%2C7560%2C65536%2C53302%2C6805%2C177%2C7777777%2C254%2C1750%2C957%2C291%2C60808%2C42220%2C183%2C1868%2C360%2C57073%2C8008%2C1923%2C7897%2C5371%2C34443%2C130%2C10%2C8453%2C480%2C1135
getting latest result
     dune: day range 22-25 (lookback_days=3, trailing_days=3)
Results available at https://dune.com/queries/3740822?trailing_days=3&lookback_days=3&list_chain_ids=185%2C252%2C7560%2C65536%2C53302%2C6805%2C177%2C7777777%2C254%2C1750%2C957%2C291%2C60808%2C42220%2C183%2C1868%2C360%2C57073%2C8008%2C1923%2C7897%2C5371%2C34443%2C130%2C10%2C8453%2C480%2C1135
getting latest result
     dune: day range 19-22 (lookback_days=6, trailing_days=3)
Results available at https://dune.com/queries/3740822?trailing_days=3&lookback_days=6&list_chain_ids=185%2C252%2C7560%2C65536%2C53302%2C6805%2C177%2C7777777%2C254%2C1750%2

In [ ]:
if not dune_df.empty:
    dune_df['source'] = 'dune'
    dune_df['dt'] = pd.to_datetime(dune_df['dt']).dt.tz_localize(None)
    dune_df['chain_id'] = dune_df['chain_id'].astype(str)
    dune_df['chain_id'] = dune_df['chain_id'].astype(str).str.replace(r'\.0$', '', regex=True)
    dune_df = dune_df[col_list]
else:
    print("No data was retrieved from Dune. The resulting DataFrame will be empty.")
    dune_df = pd.DataFrame(columns=col_list)

In [ ]:
# Verify that all elements are strings
assert dune_df['chain_id'].apply(type).eq(str).all(), "Not all elements are strings"
print(dune_df['chain_id'].dtype)

In [ ]:
# dune_df.sample(5)
print(dune_df.dtypes)

In [ ]:
print(f"ch shape: {ch.shape}")
# print(f"flip shape: {flip.shape}")
print(f"dune_df shape: {dune_df.shape}")

# Show rows with NaN values in dune_df
if not dune_df.empty:
    nan_rows = dune_df[dune_df.isna().any(axis=1)]
    if not nan_rows.empty:
        print(f"\nRows with NaN values in dune_df ({len(nan_rows)} rows):")
        print(nan_rows)
        print("\nColumns with NaN counts:")
        print(dune_df.isna().sum())
    else:
        print("\nNo NaN values found in dune_df")
else:
    print("\ndune_df is empty")

In [ ]:
# Convert chain_id and source to strings in both DataFrames
ch['chain_id'] = ch['chain_id'].astype(str).fillna('na')
ch['source'] = ch['source'].astype(str).fillna('na')
# Apply ch datatypes to dunedf
ch_dtypes = ch.dtypes.to_dict()

# Now, apply these dtypes to dune_df (only if it's not empty)
if not dune_df.empty:
    # Handle NaN values before converting to integer types
    for col, dtype in ch_dtypes.items():
        if col not in dune_df.columns:
            continue
        
        # Check if dtype is integer type
        if pd.api.types.is_integer_dtype(dtype):
            # Check if column is numeric before using isinf
            if pd.api.types.is_numeric_dtype(dune_df[col]):
                # Check if there are any NaN or infinite values
                if dune_df[col].isna().any() or np.isinf(dune_df[col]).any():
                    # Fill NaN values with 0 before converting to integer
                    dune_df[col] = dune_df[col].fillna(0).replace([np.inf, -np.inf], 0).astype(dtype)
                else:
                    # No NaN values, safe to convert directly
                    dune_df[col] = dune_df[col].astype(dtype)
            else:
                # Non-numeric column, fill NaN and convert
                dune_df[col] = dune_df[col].fillna(0).astype(dtype)
        else:
            # For non-integer types, convert normally
            dune_df[col] = dune_df[col].astype(dtype)

print("ch columns:", ch.columns)
print("dune_df columns:", dune_df.columns)

print("ch dtypes:", ch.dtypes)
print("dune_df dtypes:", dune_df.dtypes)

In [ ]:
# Combine dfs
# final_df = pd.concat([ch, dune_df])#, flip])
final_df = pd.concat([ch, dune_df], axis=0, ignore_index=True)

# Remove Dupes
final_df = final_df.drop_duplicates(subset=['chain_id','dt'], keep='first')

print(f"final_df shape: {final_df.shape}")

In [ ]:
opstack_metadata = pd.read_csv('../op_chains_tracking/outputs/chain_metadata.csv')

opstack_metadata['chain_id'] = opstack_metadata['mainnet_chain_id'].astype(str)

meta_cols = ['is_op_chain','op_based_version', 'chain_id', 'alignment','chain_name', 'display_name']

# print("Columns in opstack_metadata:", opstack_metadata.columns)
# print("Columns in opstack_metadata[meta_cols]:", opstack_metadata[meta_cols].columns)
# print("Columns in final_df:", final_df.columns)


In [ ]:
final_enriched_df = final_df.merge(opstack_metadata[meta_cols], on='chain_id', how = 'left')
final_enriched_df['alignment'] = final_enriched_df['alignment'].fillna('Other EVMs')
final_enriched_df['is_op_chain'] = final_enriched_df['is_op_chain'].fillna(False)
final_enriched_df['display_name'] = final_enriched_df['display_name'].fillna(final_enriched_df['name'])

final_enriched_df = final_enriched_df.drop(columns=['name'])

In [ ]:
# final_enriched_df = final_enriched_df.sort_values(by='dt')
# final_enriched_df[final_enriched_df['blockchain'] == 'base'].tail(5)

In [ ]:
final_enriched_df.sort_values(by=['dt','blockchain'], ascending =[False, False], inplace = True)

# final_enriched_df.to_csv('outputs/'+query_name+'.csv', index=False)

In [ ]:
final_enriched_df

In [ ]:
final_enriched_df['chain_id'] = final_enriched_df['chain_id'].astype('string').str.replace('.0', '', regex=False)
final_enriched_df['num_raw_txs'] = final_enriched_df['num_raw_txs'].astype(int)
final_enriched_df['num_success_txs'] = final_enriched_df['num_success_txs'].astype(int)
final_enriched_df['num_qualified_txs'] = final_enriched_df['num_qualified_txs'].fillna(0)
final_enriched_df['num_qualified_txs'] = final_enriched_df['num_qualified_txs'].astype(int)
final_enriched_df['sum_success_gas_used'] = final_enriched_df['sum_success_gas_used'].astype(int)
final_enriched_df['sum_qualified_gas_used'] = final_enriched_df['sum_qualified_gas_used'].fillna(0)
final_enriched_df['sum_qualified_gas_used'] = final_enriched_df['sum_qualified_gas_used'].astype(int)
# final_enriched_df = final_enriched_df.drop('row_num', axis=1)
# final_enriched_df.dtypes

In [ ]:
final_enriched_df.dtypes

In [ ]:
# Import BQ utils here (right before use) to get fresh OIDC token in CI
sys.path.append("../helper_functions")
import google_bq_utils as bqu
sys.path.pop()

# Prepare credentials before BQ operations (refreshes OIDC token if needed)
bqu.prepare_bq_credentials()

#BQ Upload
bqu.append_and_upsert_df_to_bq_table(final_enriched_df, query_name, unique_keys = ['chain_id','dt'])